In [1]:
import os
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
import string
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

import scipy as sp

[nltk_data] Downloading package punkt to C:\Users\SNEHAL
[nltk_data]     LOKESH\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 1000)

In [3]:
train_orig = pd.read_csv('E:\\Tech competitions\\Hackerearth\\ML 1\\dataset\\train.csv')
test_orig = pd.read_csv('E:\\Tech competitions\\Hackerearth\\ML 1\\dataset\\test.csv')

In [4]:
train_orig['original_author'].nunique()

1574

### With author combination 

In [235]:
train_orig["original_text"] =train_orig['original_author'] + train_orig['original_text']+' '+train_orig['original_author']
test_orig["original_text"] =train_orig['original_author'] +  test_orig['original_text']+' '+test_orig['original_author']

In [236]:
train_orig['column'] = 'train'
test_orig['column'] = 'test'

In [237]:
df_s = train_orig.groupby('original_author')['sentiment_class'].value_counts() / train_orig.groupby('original_author')['sentiment_class'].count()
df_f = df_s.reset_index(name='cpt')

In [238]:
train_orig_prob = pd.merge(train_orig, df_f,  how='inner', left_on=['original_author','sentiment_class'], right_on = ['original_author','sentiment_class'])

In [10]:
author_all = train_orig_prob.original_author.unique()
sentiment = [-1,0,1]

In [11]:
def prob(i,j):
    try:
        return float(train_orig_prob.loc[((train_orig_prob.original_author==i) & (train_orig_prob.sentiment_class==j)),'cpt'][:1])
    except:
        return 0

In [12]:
for i in author_all:
    for j in [0,-1,1]:
        train_orig_prob.loc[((train_orig_prob.sentiment_class==j) & (train_orig_prob.original_author==i)),"Neutral"] = prob(i,0)
        train_orig_prob.loc[((train_orig_prob.sentiment_class==j) & (train_orig_prob.original_author==i)),"Negative"] = prob(i,-1)
        train_orig_prob.loc[((train_orig_prob.sentiment_class==j) & (train_orig_prob.original_author==i)),"Positive"] = prob(i,1)

In [13]:
def prob_test(original_author,sentiment_class):
        return float(train_orig_prob.loc[train_orig_prob.original_author==original_author,sentiment_class][:1])

In [14]:
for i in author_all:
    test_orig.loc[test_orig.original_author==i,"Neutral"] = prob_test(i,"Neutral")
    test_orig.loc[test_orig.original_author==i,"Positive"] = prob_test(i,"Positive")
    test_orig.loc[test_orig.original_author==i,"Negative"] = prob_test(i,"Negative")

In [15]:
test_orig.isna().sum()

id                   0
original_text        0
lang                 0
retweet_count        1
original_author      0
column               0
Neutral            524
Positive           524
Negative           524
dtype: int64

In [15]:
test_orig.fillna(0.3333333,inplace=True)

In [16]:
train_orig_prob.drop(columns=["cpt"],inplace=True)

In [175]:
df_total =  pd.concat([train_orig_prob, test_orig])

In [176]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\SNEHAL
[nltk_data]     LOKESH\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [177]:
stop_words = list(stopwords.words('english'))
for i in ['against','no','not',"didn't","couldn't","shouldn't","won't"]:
    stop_words.remove(i)

In [178]:
#converting to lower case
df_total['original_text'] = df_total['original_text'].str.lower()

In [240]:
def Punctuation(string): 
  
    # punctuation marks 
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~’‘–≥®🙂∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~'''
  
    # traverse the given string and if any punctuation 
    # marks occur replace it with null 
    for x in string: 
        if x in punctuations: 
            #string = string.replace(x, " ")
            string = string.replace(x, " punctuation ") 
  
    # Print string without punctuation 
    return string

In [241]:
import re
contractions_dict = {
        'didn\'t': 'did not',
        'don\'t': 'do not',
        "aren't": "are not",
        "can't": "cannot",
        "cant": "cannot",
        "can't've": "cannot have",
        "'cause": "because",
        "could've": "could have",
        "couldn't": "could not",
        "couldn't've": "could not have",
        "didn't": "did not",
        "didnt": "did not",
        "doesn't": "does not",
        "doesnt": "does not",
        "don't": "do not",
        "dont" : "do not",
        "hadn't": "had not",
        "hadn't've": "had not have",
        "hasn't": "has not",
        "haven't": "have not",
        "he'd": "he had",
        "he'd've": "he would have",
        "he'll": "he will",
        "he's": "he is",
        "how'd": "how did",
        "how'd'y": "how do you",
        "how'll": "how will",
        "how's": "how is",
        "i'd": "i had",
        "i'd've": "i would have",
        "i'll": "i will",
        "i'm": "i am",
        "im": "i am",
        "i've": "i have",
        "isn't": "is not",
        "it'll": "it will",
        "it's": "it is",
        "let's": "let us",
        "ma'am": "madam",
        "mayn't": "may not",
        "might've": "might have",
        "mightn't": "might not",
        "must've": "must have",
        "mustn't": "must not",
        "mustn't've": "must not have",
        "needn't": "need not",
        "needn't've": "need not have",
        "oughtn't": "ought not",
        "oughtn't've": "ought not have",
        "shan't": "shall not",
        "sha'n't": "shall not",
        "shan't've": "shall not have",
        "she'd": "she had",
        "she'd've": "she would have",
        "she'll": "she will",
        "she's": "she is",
        "should've": "should have",
        "shouldn't": "should not",
        "shouldn't've": "should not have",
        "that's": "that is",
        "there's": "there is",
        "they'd": "they had",
        "they'd've": "they would have",
        "they'll": "they will",
        "they're": "they are",
        "they've": "they have",
        "to've": "to have",
        "wasn't": "was not",
        "we'd": "we had",
        "we'd've": "we would have",
        "we'll": "we will",
        "we'll've": "we will have",
        "we're": "we are",
        "we've": "we have",
        "weren't": "were not",
        "what're": "what are",
        "what's": "what is",
        "what've": "what have",
        "when've": "when have",
        "where'd": "where did",
        "where's": "where is",
        "where've": "where have",
        "who'll": "who will",
        "who's": "who is",
        "will've": "will have",
        "won't": "will not",
        "won't've": "will not have",
        "would've": "would have",
        "wouldn't": "would not",
        "wouldn't've": "would not have",
        "y'all": "you all",
        "you'll": "you will",
        "you're": "you are",
        "you've": "you have"
        }

contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()))
def expand_contractions(s, contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, s)

In [242]:
df_total['text_feature'] = df_total['original_text'].apply(Punctuation)

In [243]:
df_total['text_feature'] = df_total['text_feature'].apply(expand_contractions)

In [244]:
def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')

In [245]:
df_total['text_feature'] = df_total['text_feature'].apply(deEmojify)

In [246]:
df_total['punctuation_counts'] = df_total.text_feature.str.count('punctuation')

In [248]:
def Numbers(string): 
  
    # punctuation marks 
    punctuations = '''0123456789'''
  
    # traverse the given string and if any punctuation 
    # marks occur replace it with null 
    for x in string: 
        if x in punctuations: 
            #string = string.replace(x, " ")
            string = string.replace(x, " numbers ") 
  
    # Print string without punctuation 
    return string

In [249]:
df_total['text_feature'] = df_total['text_feature'].apply(Numbers)

In [250]:
df_total['numbers_counts'] = df_total.text_feature.str.count('numbers')

In [251]:

df_total['text_feature_words'] = df_total['text_feature'].apply(nltk.word_tokenize)

In [252]:

def remove_stops(string):
    meaningful_words = [w for w in string if not w in stop_words]
    return(meaningful_words)

In [254]:
df_total['text_feature_words_nostop'] = df_total['text_feature_words'].apply(remove_stops)

In [255]:
df_total['word_count_withstop'] = df_total['text_feature_words'].apply(len)

In [256]:
df_total['word_count_nostop'] = df_total['text_feature_words_nostop'].apply(len)

In [258]:
df_notext = df_total[['Negative','Neutral','Positive','column','original_author']]
#df_notext = df_total[['column','original_author']]

In [259]:
df_onehot = pd.DataFrame(pd.get_dummies(df_notext.original_author))
df_notext = pd.concat([df_notext,df_onehot],axis=1)
train_notext = df_notext.loc[df_notext['column'] == 'train']
test_notext = df_notext.loc[df_notext['column'] == 'test']

Negative   Neutral  Positive column original_author  -0.0609223143  -0.3350875182  -0.5266223935  -0.6227238527  -0.7265777898  -0.7512809757  0  0.0014674458  0.0160232474  0.0599994863  0.0614393786  0.0680825652  0.0819117911  0.1286903083  0.170899617  0.1932277999  0.2037180574  0.2145187008  0.2344055641  0.2684147851  0.3600148299  0.3763747794  0.3840103444  0.3875395839  0.4426014067  0.4670004957  0.5059722599  0.5155649352  0.5448525664  0.6024385152  0.6063751089  0.6416150504  0.6873825957  0.7034416192  0.7083954922  0.710938981  0.7492387603  0.7747733789  0.7990601793  0.8031074481  0.8181769252  0.8336669099  0.8474303071  0.8794361817  0.8904229163  0.9091620954  0.9382834894  0.9466307494  0.9609054742  0.963530258  0.9773241428  1  11  16  17navygirl  1968Tanja  1986Webber  1inca_scott  1ktuition  1kyancy  1lynndailey  2  20BelowMusic  215alexb  2Serenity  2brichNhealthy  3  30  30seconds  31  32sickboy  369MyName  3812  446_sk  4LaConvos  4n93lina  \
0  0.000000  1.000000  0.000000  train     BeenXXPired              0              0              0              0              0              0  0             0             0             0             0             0             0             0            0             0             0             0             0             0             0             0             0             0             0             0             0             0             0             0             0             0             0             0             0            0             0             0             0             0             0             0             0             0             0             0             0             0             0            0             0  0   0   0           0          0           0            0          0        0            0  0             0         0          0               0  0   0          0   0          0          0     0       0          0         0   
1  0.000000  1.000000  0.000000  train     BeenXXPired              0              0              0              0              0              0  0             0             0             0             0             0             0             0            0             0             0             0             0             0             0             0             0             0             0             0             0             0             0             0             0             0             0             0             0            0             0             0             0             0             0             0             0             0             0             0             0             0             0            0             0  0   0   0           0          0           0            0          0        0            0  0             0         0          0               0  0   0          0   0          0          0     0       0          0         0   
2  0.285714  0.428571  0.285714  train  FestiveFeeling              0              0              0              0              0              0  0             0             0             0             0             0             0             0            0             0             0             0             0             0             0             0             0             0             0             0             0             0             0             0             0             0             0             0             0            0             0             0             0             0             0             0             0             0             0             0             0             0             0            0             0  0   0   0           0          0           0            0          0        0            0  0             0         0          0               0  0   0          0   0          0          0     0       0          0         0   
3  0.285714  0.428571  0.285714  train  Festive

In [260]:
test_notext.drop(columns = ['column','original_author'],inplace=True)

C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [261]:
train_processed = df_total.loc[df_total['column'] == 'train']

In [262]:
test_processed = df_total.loc[df_total['column'] == 'test']

In [263]:
train_processed = train_processed.drop(columns = ['column','original_text','id','text_feature_words','text_feature_words_nostop'])

In [264]:
test_processed = test_processed.drop(columns = ['column','original_text','id','text_feature_words','text_feature_words_nostop'])

In [265]:

test_processed = test_processed.drop(columns = ['sentiment_class'])

In [206]:
from lightgbm import LGBMClassifier

In [292]:
from imblearn.over_sampling import SVMSMOTE
sm = SMOTE(sampling_strategy ='auto',k_neighbors=10,random_state=100)
sm_svm = SVMSMOTE(sampling_strategy ='minority',k_neighbors=5,random_state=500)

In [293]:
def tf(data):
    tfidf = TfidfVectorizer(ngram_range=(1, 2))
    emb = tfidf.fit_transform(data)
    return emb, tfidf

In [294]:
list_train_to_tfidf = train_processed["text_feature"].tolist()
list_train_labels = train_processed["sentiment_class"].tolist()

In [295]:
list_test_to_tfidf = test_processed["text_feature"].tolist()

In [296]:
X_train_tfidf, tfidf = tf(list_train_to_tfidf)
X_test_tfidf = tfidf.transform(list_test_to_tfidf)

In [277]:

col_to_append = ["Negative","Neutral","Positive"]

train_col_to_add_np = np.array(train_processed[col_to_append])
test_col_to_add_np = np.array(test_processed[col_to_append])
X_train_tfidf_addition_col = sp.sparse.hstack((X_train_tfidf, train_col_to_add_np))
X_test_tfidf_addition_col = sp.sparse.hstack((X_test_tfidf, test_col_to_add_np))
X_train_tfidf_addition.shape

(3235, 73972)

In [297]:
## Resampling Data

X_train_tfidf_addition_col, list_train_labels = sm_svm.fit_sample(X_train_tfidf,list_train_labels) #SVM SMOTE

In [279]:
def fit_models(algo,X_train,Y_train,X_test,submission,filename):
    print(str(algo))
    algo.fit(X_train, Y_train)
    
    prediction = algo.predict(X_train)
    print(classification_report(Y_train, prediction))
    
    submission["sentiment_class"] = algo.predict(X_test)
    file = "E:\\Tech competitions\\Hackerearth\\ML 1\\June\\"+filename+".csv"
    submission.to_csv(file,index = False)

In [280]:
from  sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [153]:
#348,8,0.1


In [298]:
# Light Gradient Boosting Regressor
lightgbm = LGBMClassifier(
        n_estimators=358, #50  #250  #350 final #750-ada  358
        num_leaves=10, #15 #15   #10 final 
        colsample_bytree=.8,
        subsample=0.6, #0.6
        max_depth=10,#7  #10 final 
        reg_alpha=.1,
        reg_lambda=.1,
        min_split_gain=0.06,#0.06
        min_child_weight=0.001
    )

In [299]:
#fit_models(lightgbm, X_trin_tfidf_addition_col, list_train_labels, X_test_tfidf_addition_col,test_orig[['id']],"ligbm_res_2" )
fit_models(lightgbm, X_train_tfidf_addition_col, list_train_labels, X_test_tfidf,test_orig[['id']],"ligbm_res_4" ) #Final 44.01

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.8,
               importance_type='split', learning_rate=0.1, max_depth=10,
               min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.06, n_estimators=358, n_jobs=-1, num_leaves=10,
               objective=None, random_state=None, reg_alpha=0.1, reg_lambda=0.1,
               silent=True, subsample=0.6, subsample_for_bin=200000,
               subsample_freq=0)
              precision    recall  f1-score   support

        -1.0       0.99      0.79      0.87       769
         0.0       0.85      0.99      0.92      1701
         1.0       0.99      0.92      0.95      1701

    accuracy                           0.92      4171
   macro avg       0.94      0.90      0.91      4171
weighted avg       0.93      0.92      0.92      4171



C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
